In [ ]:
from autogen_core.models import ChatCompletionClient

In [9]:
config = {
    "provider": "OpenAIChatCompletionClient",
    "config": {
        "model": "qwen2-vl-2b-instruct",
        "base_url": "http://127.0.0.1:1234/v1",
        "api_key": "lm-studio",
        "model_info": {
            "name": "qwen2-vl-2b-instruct",
            "family": "openai",
            "supports_tool_calling": False,
            "supports_json_mode": False,
            "structured_output": True,
            "json_output": True,
            "function_calling": True,
            "vision": True,
        }
    }
}

client = ChatCompletionClient.load_component(config)

## Sequential Flow

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [7]:
# Create the writer agent
writer = AssistantAgent("writer", model_client=client,
                        system_message="Draft a short paragraph on climate change.")

# Create the reviewer agent
reviewer = AssistantAgent("reviewer", model_client=client,
                          system_message="Review the draft and suggest improvements.")

# Build the graph
builder = DiGraphBuilder()
builder.add_node(writer).add_node(reviewer)
builder.add_edge(writer, reviewer)

# Build and validate the graph
graph = builder.build()

# Create the flow
flow = GraphFlow([writer, reviewer], graph=graph)

In [8]:
# Use `asyncio.run(...)` and wrap the below in a async function when running in a script.
stream = flow.run_stream(task="Write a short paragraph about climate change.")
async for event in stream:  # type: ignore
    print(event)
# Use Console(flow.run_stream(...)) for better formatting in console.

source='user' models_usage=None metadata={} content='Write a short paragraph about climate change.' type='TextMessage'
source='writer' models_usage=RequestUsage(prompt_tokens=31, completion_tokens=127) metadata={} content='Climate change is one of the most pressing issues of our time, with far-reaching consequences for our planet and its inhabitants. The burning of fossil fuels, deforestation, and other human activities have led to a significant increase in greenhouse gas emissions, resulting in rising temperatures, melting ice caps, and more extreme weather events. If left unchecked, climate change will continue to worsen, causing devastating impacts on ecosystems, economies, and human societies worldwide. It is essential that we take immediate action to reduce our carbon footprint, invest in renewable energy, and adopt sustainable practices to mitigate the effects of climate change and create a more resilient future for all.' type='TextMessage'
source='reviewer' models_usage=RequestU

## Parallel Flow with Join

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import MultiModalMessage
from autogen_core import CancellationToken, Image
from pathlib import Path
import asyncio


# System prompt cho grader
system_prompt = """
You are an experienced teacher.
Your job is to independently evaluate whether the student's answer is correct or incorrect, based on the target answer provided.
Please read both the student's answer and the target answer carefully.
When you complete your evaluation, explain your reasoning clearly and concisely.
Use the following response format:
[Correct], because ...
or
[Incorrect], because ...
"""

system_vqa = """
You are a visual question answering assistant.

Given an image and a question, your task is:
1. Try to answer the question directly based on the visual content.
2. If the question cannot be confidently answered, DO NOT guess. Instead, explain why you are uncertain and describe what is missing in the image.
3. Always justify your answer using visual evidence from the image.

Use this response format:
Answer: <your answer or '[Answer Failed]'>
Explanation: <your reasoning>
"""

vqa_with_explanation_agent = AssistantAgent(
    "vqa_with_explanation_agent",
    model_client=client,
    system_message=system_vqa
)


grader1 = AssistantAgent("grader1", model_client=client,
                         system_message=system_prompt)
grader2 = AssistantAgent("grader2", model_client=client,
                         system_message=system_prompt)
grader3 = AssistantAgent("grader3", model_client=client,
                         system_message=system_prompt)


voter = AssistantAgent(
    "voter",
    model_client=client,
    system_message="""
You are a voting assistant.

Your task is to:
1. Wait until you have received **3 evaluations** from the graders.
2. Read all 3 evaluations.
3. Determine whether the majority voted [Correct] or [Incorrect].
4. Justify the majority vote briefly based on their explanations.

Respond in the following format:
Majority vote: [Correct or Incorrect]
Justification: <short explanation>

Only respond once after receiving all 3 evaluations.
"""
)
# Xây dựng đồ thị workflow
builder = DiGraphBuilder()
builder.add_node(vqa_with_explanation_agent)
builder.add_node(grader1).add_node(grader2).add_node(grader3)
builder.add_node(voter)

# VQA agent → các grader
builder.add_edge(vqa_with_explanation_agent, grader1)
builder.add_edge(vqa_with_explanation_agent, grader2)
builder.add_edge(vqa_with_explanation_agent, grader3)

# Grader → Voter
builder.add_edge(grader1, voter)
builder.add_edge(grader2, voter)
builder.add_edge(grader3, voter)

graph = builder.build()

flow = GraphFlow(
    participants=builder.get_participants(),
    graph=graph,
)

# Đọc ảnh từ file
image_path = Path("cat.jpg")  # Đổi đường dẫn tới ảnh phù hợp
image = Image.from_file(image_path)

# Tạo message đầu vào dạng multimodal
message = MultiModalMessage(
    content=[
        "Question: What is the animal doing?",
        image
    ],
    source="user"
)

cancellation_token = CancellationToken()
stream = flow.run_stream(          
        task=[message],                 
        cancellation_token=cancellation_token,
    )
await Console(stream) 

client.close()

---------- MultiModalMessage (user) ----------
Question: What is the animal doing?
<image>
---------- TextMessage (vqa_with_explanation_agent) ----------
The cat is sitting and looking at the camera.
---------- TextMessage (grader1) ----------
Correct, because the target answer specifies that the cat is sitting and looking at the camera.
---------- TextMessage (grader2) ----------
Correct, because the target answer describes the cat's position and activity.
---------- TextMessage (grader3) ----------
[Incorrect], because the target answer does not mention the cat's position or activity.
---------- TextMessage (voter) ----------
Majority vote: Correct

Justification: The evaluation states that the cat is "sitting" and "looking at the camera," which matches the information provided in the image. The evaluation also mentions that the cat is sitting on a wall, which is consistent with the image. Therefore, the majority of the evaluations agree with the correct answer.
---------- StopMessag

<coroutine object BaseOpenAIChatCompletionClient.close at 0x000001B21959B610>